## Fake NEws Classificer USsing Bidirectional LSTM

In [1]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd



In [2]:
#Loading data
df=pd.read_csv('train.csv',engine='python',error_bad_lines=False)

<ipython-input-2-772ef6e293b2>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('train.csv',engine='python',error_bad_lines=False)
Skipping line 4206: unexpected end of data


In [3]:

df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id          0
title     113
author    416
text        9
label       0
dtype: int64

In [5]:
df.shape

(4204, 5)

In [6]:
#Deleting raws which has null values
df=df.dropna()

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
## get the independent features
X=df.drop('label',axis=1)

In [10]:
#Target variable
y=df['label']

In [11]:
## Check whether dataset is balanced or not
y.value_counts()

0    2056
1    1619
Name: label, dtype: int64

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.13.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [16]:
#Taking a copy of data
messages=X.copy()

In [17]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
#Dowloading stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
#Resetting index as we deleted some of the raws while dropping null values
messages.reset_index(inplace=True)

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

## Embedding Representation

In [15]:
## vocabulary size ie number of unique variable/words in corpus
voc_size=5000

In [23]:
# Converting words to one hot representation
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2577, 1129, 1109, 10, 43, 102, 4374, 3811, 894, 3233],
 [891, 4088, 2363, 3237, 1158, 4432, 2484],
 [4364, 4298, 4025, 2994],
 [4778, 356, 4260, 3948, 1404, 4143],
 [3833, 1158, 1024, 784, 317, 97, 1158, 862, 1568, 1673],
 [406,
  3347,
  873,
  364,
  3951,
  887,
  2259,
  4645,
  1098,
  1141,
  3373,
  1160,
  363,
  1119,
  2484],
 [1396, 702, 3428, 2804, 4929, 2484, 96, 1143, 2308, 3076, 4496],
 [3190, 2496, 3352, 2738, 970, 4818, 887, 3505, 2308, 3076, 4496],
 [4287, 2858, 4601, 340, 4715, 3712, 1090, 3352, 887, 3485],
 [2917, 2716, 1176, 766, 3359, 3308, 3031, 1035],
 [231, 1099, 1741, 3071, 2440, 1523, 1649, 4963, 1804, 1934, 3170],
 [3948, 3875, 43, 3712, 887, 970],
 [630, 1623, 1179, 3500, 4835, 3642, 2765, 2772, 1206],
 [3362, 1040, 3699, 1300, 1165, 626, 4410, 2308, 3076, 4496],
 [3208, 2123, 281, 2831, 2291, 2308, 3076, 4496],
 [1254, 4146, 4297, 3217, 3657, 1343, 2581, 4324, 2652, 2831],
 [3481, 4166, 4088],
 [4810, 4112, 4712, 526, 887, 2437, 2486, 2484],
 [2902, 678,

In [24]:
# Defining sentence maximum length expected
sent_length=20
# Add padding if sentence length is less than 20 this is inorder to make a constant length of input to network
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 3811,  894, 3233],
       [   0,    0,    0, ..., 1158, 4432, 2484],
       [   0,    0,    0, ..., 4298, 4025, 2994],
       ...,
       [   0,    0,    0, ...,  128, 3426, 2127],
       [   0,    0,    0, ..., 3070,  749, 2484],
       [   0,    0,    0, ..., 1380, 3014, 3951]], dtype=int32)

In [25]:
## Creating model
embedding_vector_features=40 # feature/word representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [28]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
77/77 [==============================] - 19s 122ms/step - loss: 0.4471 - accuracy: 0.7665 - val_loss: 0.2521 - val_accuracy: 0.8986
Epoch 2/10
77/77 [==============================] - 3s 40ms/step - loss: 0.1479 - accuracy: 0.9444 - val_loss: 0.2437 - val_accuracy: 0.9035
Epoch 3/10
77/77 [==============================] - 2s 20ms/step - loss: 0.0604 - accuracy: 0.9801 - val_loss: 0.2802 - val_accuracy: 0.8920
Epoch 4/10
77/77 [==============================] - 2s 22ms/step - loss: 0.0170 - accuracy: 0.9968 - val_loss: 0.4932 - val_accuracy: 0.8821
Epoch 5/10
77/77 [==============================] - 1s 16ms/step - loss: 0.0100 - accuracy: 0.9988 - val_loss: 0.4792 - val_accuracy: 0.8821
Epoch 6/10
77/77 [==============================] - 1s 12ms/step - loss: 0.0064 - accuracy: 0.9988 - val_loss: 0.5094 - val_accuracy: 0.8862
Epoch 7/10
77/77 [==============================] - 1s 17ms/step - loss: 0.0034 - accuracy: 0.9996 - val_loss: 0.5502 - val_accuracy: 0.8879
Epoch 8/10


In [29]:
y_pred=model.predict(X_test)

38/38 [==============================] - 1s 3ms/step


In [30]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [31]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[617,  62],
       [ 78, 456]])

In [32]:
print(accuracy_score(y_test,y_pred))

0.8845836768342952


In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90       679
           1       0.88      0.85      0.87       534

    accuracy                           0.88      1213
   macro avg       0.88      0.88      0.88      1213
weighted avg       0.88      0.88      0.88      1213

